In [18]:
import ee
import pandas as pd

# Initialize the Earth Engine library
ee.Initialize(project='pranavkoka123')

# Define a simple region of interest (a small area)
geometry = ee.Geometry.Rectangle([79.0333, 30.7333, 79.0667, 30.8000])

# Get a single Landsat 8 image for a specific date
image = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
            .filterDate('2015-01-01', '2015-01-31') \
            .filterBounds(geometry) \
            .first()

# Calculate NDSI function
def calculate_ndsi(image):
    # NDSI = (Green - SWIR) / (Green + SWIR)
    green = image.select('B3')  # Green band
    swir = image.select('B6')  # SWIR band
    ndsi = green.subtract(swir).divide(green.add(swir)).rename('NDSI')
    return image.addBands(ndsi)

# Apply NDSI calculation
image_ndsi = calculate_ndsi(image)

# Apply threshold to NDSI values
def apply_ndsi_threshold(image):
    # NDSI threshold: set values less than 0.4 to 0
    ndsi = image.select('NDSI')
    ndsi_thresholded = ndsi.where(ndsi.lt(0.4), 0).rename('NDSI_thresholded')
    return image.addBands(ndsi_thresholded)

# Apply the threshold to the NDSI
image_ndsi_thresholded = apply_ndsi_threshold(image_ndsi)


# Function to extract snow/ice confidence, snow confidence, cloud, and cloud shadow confidence from QA band
def extract_confidence_bits(image):
    qa = image.select('QA_PIXEL')  # QA_PIXEL contains the quality band

    # Extract Snow/Ice Confidence (Bits 12-13)
    snow_ice_confidence = qa.rightShift(12).bitwiseAnd(3).rename('Snow_Ice_Confidence')

    # Extract Snow Confidence (Bit 5)
    snow_confidence = qa.rightShift(5).bitwiseAnd(1).rename('Snow_Confidence')

    # Extract Cloud Confidence (Bit 3)
    cloud_confidence = qa.rightShift(3).bitwiseAnd(1).rename('Cloud_Confidence')

    # Extract Cloud Shadow Confidence (Bit 4)
    cloud_shadow_confidence = qa.rightShift(4).bitwiseAnd(1).rename('Cloud_Shadow_Confidence')

    return image.addBands([snow_ice_confidence, snow_confidence, cloud_confidence, cloud_shadow_confidence])

# Apply the bit extraction
image_with_confidence = extract_confidence_bits(image_ndsi_thresholded)

# Reduce the region to get pixel values for NDSI (both thresholded and original), Snow/Ice Confidence, Snow Confidence, Cloud Confidence, and Cloud Shadow Confidence
def get_pixel_values(image, geometry):
    # Create a dictionary of band values
    reduced_image = image.reduceRegion(
        reducer=ee.Reducer.toList(), 
        geometry=geometry, 
        scale=30
    )
    return reduced_image

# Get the pixel values from the image
pixel_values = get_pixel_values(image_with_confidence, geometry)

# Convert the data into a DataFrame
ndsi_values = pixel_values.get('NDSI').getInfo()  # Original NDSI values
ndsi_thresholded_values = pixel_values.get('NDSI_thresholded').getInfo()  # Thresholded NDSI values
snow_ice_conf_values = pixel_values.get('Snow_Ice_Confidence').getInfo()
snow_conf_values = pixel_values.get('Snow_Confidence').getInfo()
cloud_conf_values = pixel_values.get('Cloud_Confidence').getInfo()
cloud_shadow_conf_values = pixel_values.get('Cloud_Shadow_Confidence').getInfo()

# Create the DataFrame
df_jan = pd.DataFrame({
    'NDSI': ndsi_values,  # Original NDSI
    'NDSI_thresholded': ndsi_thresholded_values,  # Thresholded NDSI
    'Snow/Ice Confidence': snow_ice_conf_values,
    'Snow Confidence': snow_conf_values,
    'Cloud Confidence': cloud_conf_values,
    'Cloud Shadow Confidence': cloud_shadow_conf_values
})

# Display the DataFrame
print(df_jan)


           NDSI  NDSI_thresholded  Snow/Ice Confidence  Snow Confidence  \
0      0.661263          0.661263                    1                0   
1      0.617613          0.617613                    1                0   
2      0.580335          0.580335                    1                0   
3      0.570394          0.570394                    1                0   
4      0.600027          0.600027                    1                0   
...         ...               ...                  ...              ...   
26262  0.693757          0.693757                    1                0   
26263  0.699874          0.699874                    1                0   
26264  0.706850          0.706850                    1                0   
26265  0.707879          0.707879                    1                0   
26266  0.701454          0.701454                    1                0   

       Cloud Confidence  Cloud Shadow Confidence  
0                     1                        0

In [2]:
# Create a new image where snow confidence 1 -> white (255), and snow confidence 0 -> black (0)
def create_snow_confidence_image(image):
    # Select the Snow Confidence band (assumes 1 means snow, 0 means no snow)
    snow_confidence = image.select('Snow_Confidence')
    
    # Create a mask where Snow Confidence is 1 -> white (255), otherwise black (0)
    snow_confidence_mask = snow_confidence.eq(1).multiply(255).rename('Snow_Confidence_Mask')
    
    return snow_confidence_mask

# Apply the mask creation function
snow_confidence_image = create_snow_confidence_image(image_with_confidence)

# Define export task to a folder named 'cptest' in Google Drive
task = ee.batch.Export.image.toDrive(**{
    'image': snow_confidence_image,  # Export the Snow Confidence Mask
    'description': 'Snow_Confidence_White_Black',
    'folder': 'cptest',  # Folder name in Google Drive
    'scale': 30,  # Pixel resolution in meters
    'region': geometry.getInfo()['coordinates'],  # Area of interest
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13  # Ensure that max pixels limit is not exceeded
})

# Start the export task
task.start()

print('Export task started. Check your Google Drive folder "cptest" for the snow confidence result.')


Export task started. Check your Google Drive folder "cptest" for the snow confidence result.


In [6]:
# Create an image where NDSI > 0.8 is white (255) and NDSI <= 0.8 is black (0)
def create_ndsi_threshold_image(image):
    # Select the thresholded NDSI band
    ndsi_thresholded = image.select('NDSI_thresholded')
    
    # Create a mask where NDSI > 0.8 -> white (255), otherwise black (0)
    ndsi_mask = ndsi_thresholded.gt(0.8).multiply(255).rename('NDSI_White_Black')
    
    return ndsi_mask

# Apply the mask creation function
ndsi_threshold_image = create_ndsi_threshold_image(image_ndsi_thresholded)

# Define export task to a folder named 'cptest' in Google Drive
task = ee.batch.Export.image.toDrive(**{
    'image': ndsi_threshold_image,  # Export the NDSI threshold mask
    'description': 'NDSI_White_Black_0.8',
    'folder': 'cptest',  # Folder name in Google Drive
    'scale': 30,  # Pixel resolution in meters
    'region': geometry.getInfo()['coordinates'],  # Area of interest
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13  # Ensure that max pixels limit is not exceeded
})

# Start the export task
task.start()

print('Export task started. Check your Google Drive folder "cptest" for the NDSI white-black result.')


Export task started. Check your Google Drive folder "cptest" for the NDSI white-black result.


In [14]:
# Create a gradual grayscale image from NDSI values
def create_ndsi_grayscale_image(image):
    # Select the NDSI band
    ndsi = image.select('NDSI')
    
    # Rescale NDSI values from [-1, 1] to [0, 255] for grayscale
    ndsi_rescaled = ndsi.add(1).multiply(127.5).rename('NDSI_Grayscale')
    
    return ndsi_rescaled

# Apply the grayscale creation function
ndsi_grayscale_image = create_ndsi_grayscale_image(image_ndsi)

# Define export task to a folder named 'cptest' in Google Drive
task = ee.batch.Export.image.toDrive(**{
    'image': ndsi_grayscale_image,  # Export the NDSI grayscale image
    'description': 'NDSI_Grayscale_apr',
    'folder': 'cptest',  # Folder name in Google Drive
    'scale': 30,  # Pixel resolution in meters
    'region': geometry.getInfo()['coordinates'],  # Area of interest
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13  # Ensure that max pixels limit is not exceeded
})

# Start the export task
task.start()

print('Export task started. Check your Google Drive folder "cptest" for the NDSI grayscale result.')


Export task started. Check your Google Drive folder "cptest" for the NDSI grayscale result.


In [8]:
task = ee.batch.Export.image.toDrive(**{
    'image': image.select(['B2', 'B3', 'B4']),  # Export original image
    'description': 'Original_Image',
    'folder': 'cptest',  # Folder name in Google Drive
    'scale': 30,  # Pixel resolution in meters
    'region': geometry.getInfo()['coordinates'],  # Area of interest
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13  # Ensure that max pixels limit is not exceeded
})

task.start()